In [ ]:
/* --------------------
   Case Study Questions
   --------------------*/

--1. What is the total amount each customer spent at the restaurant?
SELECT
    customer_id,
    SUM(total_amt_spent) AS total_amount_spent
FROM (
    SELECT
        s.customer_id,
        COUNT(s.product_id) * m.price AS total_amt_spent
    FROM dannys_diner.sales AS s
    INNER JOIN dannys_diner.menu AS m ON s.product_id = m.product_id
    WHERE s.product_id = m.product_id
    GROUP BY s.customer_id, m.price
) AS amt_spent
GROUP BY customer_id;

In [ ]:
-- 2. How many days has each customer visited the restaurant?
SELECT
	customer_id,
    COUNT(DISTINCT order_date) AS days_visited
FROM dannys_diner.sales
GROUP BY customer_id
ORDER BY days_visited DESC;

In [ ]:
 3. What was the first item from the menu purchased by each customer?
SELECT customer_id, product_name
FROM (
    SELECT 
        s.customer_id, 
        m.product_name,
        ROW_NUMBER() OVER (PARTITION BY s.customer_id ORDER BY s.order_date) AS row_num
    FROM 
        dannys_diner.sales AS s
    INNER JOIN 
        dannys_diner.menu AS m ON s.product_id = m.product_id
) AS numbered_rows
WHERE row_num = 1;

In [ ]:
-- 4. What is the most purchased item on the menu and how many times was it purchased by all customers?
SELECT m.product_name, COUNT(s.product_id) AS purchase_count
FROM dannys_diner.menu as m
INNER JOIN dannys_diner.sales as s
ON m.product_id = s.product_id
GROUP BY m.product_name
ORDER BY purchase_count DESC;

In [ ]:
--5. Which item was the most popular for each customer?
WITH RankedProducts AS (
    SELECT
        s.customer_id,
        m.product_name,
  		COUNT(s.product_id),
        RANK() OVER (PARTITION BY s.customer_id ORDER BY COUNT(s.product_id) DESC) AS rnk
    FROM
        dannys_diner.sales AS s
    INNER JOIN
        dannys_diner.menu AS m ON s.product_id = m.product_id
    GROUP BY
        s.customer_id, m.product_name
)
SELECT
    customer_id,
    product_name,
    count
FROM
    RankedProducts
WHERE
    rnk = 1;

In [ ]:
-- 6. Which item was purchased first by the customer after they became a member?
WITH new_mem AS (
  SELECT 
    s.customer_id, 
    m.product_name, 
    s.order_date,
    ROW_NUMBER() OVER (PARTITION BY s.customer_id ORDER BY s.order_date) AS row_num
  FROM 
    dannys_diner.sales AS s
  INNER JOIN 
    dannys_diner.menu AS m ON s.product_id = m.product_id
  INNER JOIN 
    dannys_diner.members AS mem ON s.customer_id = mem.customer_id
  WHERE 
    s.customer_id = mem.customer_id AND s.order_date >= mem.join_date
  GROUP BY 
    s.customer_id, m.product_name, s.order_date
)
SELECT 
  customer_id, 
  product_name AS mem_first_product
FROM 
  new_mem
WHERE 
  row_num = 1;

In [ ]:
--7. Which item was purchased just before the customer became a member?
WITH non_member AS (
    SELECT 
        s.customer_id, 
        s.order_date, 
        s.product_id,
        m.product_name, 
        m.price,
        ROW_NUMBER() OVER (PARTITION BY s.customer_id ORDER BY s.order_date DESC) AS last_date
    FROM
        dannys_diner.sales AS s
    INNER JOIN 
        dannys_diner.menu AS m ON s.product_id = m.product_id
    LEFT JOIN 
        dannys_diner.members AS mem ON s.customer_id = mem.customer_id
    WHERE
        (
            s.customer_id IN (SELECT customer_id FROM dannys_diner.members) 
            AND s.order_date < mem.join_date
        )
        OR 
        (
            s.customer_id NOT IN (SELECT customer_id FROM dannys_diner.members)
        )
)  
SELECT 
	customer_id, product_name
FROM
	non_member
WHERE
	last_date = 1

In [ ]:
--8. What is the total items and amount spent for each member before they became a member?
SELECT
	customer_id, SUM(amount_spent), SUM(items_bought)
FROM (
  SELECT
  	customer_id,
  	COUNT (product_id)* price AS amount_spent,
  	COUNT (product_id) AS items_bought  	
  FROM 
  	non_member
  GROUP BY
  	customer_id, price
  ORDER BY
  	amount_spent DESC
)AS subquery
GROUP BY customer_id

In [ ]:
9.  If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?
SELECT 
    customer_id,
    SUM(
        CASE
            WHEN s.product_id = m.product_id AND m.product_name = 'sushi'
                THEN 2 * 10  -- Assuming you want to add 20 points
            WHEN s.product_id = m.product_id AND m.product_name <> 'sushi'
                THEN 10      -- Assuming you want to add 10 points
            ELSE 0
        END
    ) AS customer_points
FROM
    dannys_diner.sales AS s
INNER JOIN
    dannys_diner.menu AS m ON s.product_id = m.product_id
GROUP BY
    customer_id;

In [ ]:
-- 10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

WITH first_week AS (
    SELECT 
        s.order_date,
        s.customer_id
    FROM 
        dannys_diner.sales AS s
    INNER JOIN
        dannys_diner.members AS mem ON mem.customer_id = s.customer_id
    WHERE 
        s.order_date BETWEEN mem.join_date AND (mem.join_date + integer '6')
)

SELECT
    s.customer_id,
    SUM (
        CASE 
            WHEN s.order_date IN (SELECT order_date FROM first_week)
                THEN 2 * 10
            ELSE 10
        END
    ) AS member_points
FROM 
    first_week AS f
INNER JOIN 
    dannys_diner.sales AS s ON f.customer_id = s.customer_id
WHERE
    DATE_PART('month', s.order_date) = 1
GROUP BY
    s.customer_id;